In [101]:
import urllib.request
import re
from bs4 import BeautifulSoup
from lxml import html
import pandas as pd
# import xmltojson
import json
import requests

In [102]:
def range_price(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 \
        (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'
    }
        
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "lxml")
    lowend = soup.find_all("span", {"class":"value", "itemprop":"lowprice", "content": True})
    highend = soup.find_all("span", {"class":"value", "itemprop":"highprice", "content": True})
    quantityList = []
    rangePrice = ""

    for i in lowend:
        rangePrice+=(i["content"])
        rangePrice+=" - "
        break

    for i in highend:
        rangePrice+=(i["content"])
        break

    return rangePrice

In [103]:
def scrape_handguns_helper(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 \
        (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'
    }
    url+= "=undefined&start=0&sz=100"
        
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "lxml")
    ammo = soup.find_all("div", class_="pdp-link")
    ammoPrice = soup.find_all("div",  class_="tile-body")
    names = []
    prices = []
    actual_prices = []

    for i in ammo:
        names.append(i.find("span").text)

    for i in ammoPrice:
        # if the product has a range of prices go here
        if '<span class="range">' in str(i):
            x = i.find("a", {"class": "link", "href":True})
            prices.append(x["href"])

        # if the product has a slashed price, go here
        else:
            x = (i.find("span", class_="sales"))
            prices.append(x.find("span", {"class":"value", "content": True}))


    # gather product detail links
    soup_links = soup.find_all("div", class_="pdp-link")
    links = []
    for i in soup_links:
        aLink = re.findall("(?P<url>https?://[^\s]+)", str(i))
        x = (str(aLink[0]).replace('">', "").replace('"', ""))
        links.append(x)



    # add prices in the prices list
    for i in prices:
        if "http" in i:
            rp = range_price(i)
            actual_prices.append(rp)

        else:
            htmlPriceTag = str(i)
            splitHtmlPriceTag = htmlPriceTag.split(" ")
            p =  splitHtmlPriceTag[2].strip('content="')
            actual_prices.append(str(p))
   
    try:
        productList = pd.DataFrame(
            {"name": names, "price": actual_prices, "link": links}
        )
        return productList

    except:
        print("something went wrong")

In [104]:
link = "https://www.cheaperthandirt.com/shop-by?cgid=80&searchBy=Gauge"

def scrape_shotguns(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 \
        (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'
    }
        
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "lxml")
    sections = soup.find_all("div", class_="brand-link-list")

    # get caliber number set
    caliberNumbers = []
    x = soup.find_all('div', class_="refinements")
    for i in x:
        cn = i.find_all("div", {"id": True})
        for j in cn:
            caliberNumbers.append((j["id"]))

    dirtyLinks = []  
    calibers = []
    for i in sections:
        dirtyLinks.append(i.find_all("a"))
        x = (i.find_all("a"))
        for j in x:
            c = (j.find("span").text)
            calibers.append(c)


    caliberNumberDF = pd.DataFrame(
        {"number": caliberNumbers, "section": dirtyLinks}
    )
    
    arrayOfLinks = []
    index = 0
    for i in caliberNumberDF["section"]:
        linkToCaliberList = []  # links
        for j in i:
            aLink = re.findall("(?P<url>https?://[^\s]+)", str(j))
            arrayOfLinks.append(str(aLink[0]).replace('">', ""))

    arrayOfDicts = []
    index = 0
    
    for i in arrayOfLinks:
        productDict = {"caliber": [], "names": [],'caliber': [], "price": [], "link": []}
        productDF = scrape_handguns_helper(i.replace(";", "&"))

        aNameListD = {}
        aNameListL = []

        aPriceList = []
        aUrlList = []

        for name in productDF["name"]:
            aNameListL.append(name)
        aNameListD["name"] = aNameListL
        productDict["names"] = {aNameListL}

        for link in productDF["link"]:
            aUrlList.append(link)
        productDict["link"] = aUrlList

        for price in productDF["price"]:
            aPriceList.append(price)
        productDict["price"] = aPriceList

        productDict["caliber"] = calibers[index]
        index += 1

        arrayOfDicts.append(productDict)
        
    return arrayOfDicts


In [105]:
ammoDF = scrape_shotguns(link)

TypeError: unhashable type: 'list'

In [ ]:
for i in ammoDF:
    print(i)
    print()

{'caliber': '10 Gauge', 'names': ['Federal 10 Gauge Shotshell 5 Rounds 3.5" 00 Buck Copper Plated 18 Pellets', 'Federal Speed Shok Waterfowl Steel 10 Gauge Ammunition 3-1/2" T Steel Shot 1-1/2...', 'Hevi-Shot Hevi-Metal Longer Range Ammunition 10 Gauge 3-1/2" BBB Shot 1-3/4 oz L...', 'Hevi-Shot Hevi-Metal Longer Range Ammunition 10 Gauge 3-1/2" BB Shot 1-3/4 oz Le...', 'Federal Power-Shok 10ga 3-1/2" Rifled Slug 1-3/4oz 5 Rnd Box'], 'price': ['19.89', '54.66', '78.83', '78.83', '25.31'], 'link': ['https://www.cheaperthandirt.com/federal-10-gauge-shotshell-5-rounds-3.5-00-buck-copper-plated-18-pellets/FC-029465028343.html', 'https://www.cheaperthandirt.com/federal-speed-shok-waterfowl-steel-10-gauge-ammunition-3-12-t-steel-shot-1-12-oz-1450-fps/FC-029465027933.html', 'https://www.cheaperthandirt.com/hevi-shot-hevi-metal-longer-range-ammunition-10-gauge-3-12-bbb-shot-1-34-oz-lead-free-shot-1350-fps/FC-816383002681.html', 'https://www.cheaperthandirt.com/hevi-shot-hevi-metal-longer-range-

In [ ]:
from pathlib import Path
import os, shutil, glob
rel = "jsonFiles/cheaperThanDirt/ammo/shotgun"
cwd = Path.cwd()
currentPath = str(Path.cwd())+""
aCount = 0
savePath = str(cwd).replace("pyScripts/cheaperThanDirt", rel)
index = 0

swiftTransferDict = {"results": []}
for i in ammoDF:
    with open(os.path.join(savePath, f"CTDHShotgunAmmo{index}.json"), "w") as fp:
        json.dump(i, fp)
        index += 1


rel = '/crispy-computing-app/crispy-computing-app/jsonFiles/shotgun'
cwd = Path.cwd()
currentPath = str(Path.cwd())+''
savePath = str(cwd).replace("pyScripts/cheaperThanDirt", rel)
index = 0
for i in ammoDF:
    with open(os.path.join(savePath, f"CTDShotgunAmmo{index}.json"), "w") as fp:
        json.dump(i, fp)
        index += 1